In [12]:
# Configuration
from dotenv import dotenv_values
config = dotenv_values(".env")

json_filename = f"{config['LEARN_COURSE_ID']}.json"

# https://developer.blackboard.com/portal/displayApi
api_token = f"{config['LEARN_API_URL']}/v1/oauth2/token"
api_course = f"{config['LEARN_API_URL']}/v3/courses/courseId:{config['LEARN_COURSE_ID']}"
api_course_content = f"{config['LEARN_API_URL']}/v1/courses/courseId:{config['LEARN_COURSE_ID']}/contents"
api_course_content_children = lambda id: f"{api_course_content}/{id}/children"

In [13]:
# Authentication
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

client = BackendApplicationClient(client_id=config['LEARN_CLIENT_ID'])
with OAuth2Session(client=client, auto_refresh_url=api_token) as session:
    token = session.fetch_token(token_url=api_token, client_id=config['LEARN_CLIENT_ID'], client_secret=config['LEARN_CLIENT_SECRET'])

In [14]:
# Download course details JSON
course_details = session.get(api_course).json()

In [15]:
# Download course content JSON
course_content = session.get(api_course_content).json()['results']

def get_children_recursively(element):
    if 'children' not in element and 'hasChildren' in element and element['hasChildren'] == True:
        element['children'] = session.get(api_course_content_children(element['id'])).json()['results']
        for child in element['children']:
            get_children_recursively(child)

for element in course_content:
    get_children_recursively(element)

In [16]:
# Write course details and content JSON to local file
from json import dump

course_json = { 'details': course_details, 'content': course_content }

with open(file=json_filename, mode="w", encoding="utf-8") as f:
    dump(course_json, f, ensure_ascii=False)